<a href="https://colab.research.google.com/github/lostb1d/LST/blob/main/LST_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
#loading the Earth Engine API, GEE Authenticating and Loading Project
import ee
import folium
from IPython.display import Image
import geemap

In [3]:
#initiating gee
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='lst-nou')


In [4]:
#loading the shapefile from the above mounted google drive

shape_path = '/content/drive/MyDrive/LST/Shapefiles/Dang/Bajura_Local.shp'
print(shape_path)
shp = geemap.shp_to_ee(shape_path)


/content/drive/MyDrive/LST/Shapefiles/Dang/Bajura_Local.shp
No module named 'pycrs'


In [6]:
# Define a region of interest
roi = ee.Geometry.Polygon( [
            [
              82.21808522925141,
              28.336850451368647
            ],
            [
              81.91592954680112,
              28.126683552796777
            ],
            [
              81.90291805808266,
              27.841984062675692
            ],
            [
              82.443617700363,
              27.61804029507458
            ],
            [
              82.68939026503506,
              27.633411121765874
            ],
            [
              82.92215134098836,
              27.711512766230356
            ],
            [
              82.69328913525277,
              28.197409554609237
            ],
            [
              82.50245396738859,
              28.300565811038936
            ],
            [
              82.21808522925141,
              28.336850451368647
            ]
          ])



In [22]:
# prompt: create the dataframe with the startdate and end date from 2004 to 2024

import pandas as pd

# Create a list of years from 2004 to 2024
years = list(range(2004, 2025))

# Create a list of start dates and end dates
start_dates = []
end_dates = []
for year in years:
    start_dates.append(str(year) + '-03-01')
    end_dates.append(str(year) + '-06-30')

# Create a DataFrame with the start and end dates
df = pd.DataFrame({
    'start_date': start_dates,
    'end_date': end_dates
})

# Print the DataFrame
print(df)


    start_date    end_date
0   2004-03-01  2004-06-30
1   2005-03-01  2005-06-30
2   2006-03-01  2006-06-30
3   2007-03-01  2007-06-30
4   2008-03-01  2008-06-30
5   2009-03-01  2009-06-30
6   2010-03-01  2010-06-30
7   2011-03-01  2011-06-30
8   2012-03-01  2012-06-30
9   2013-03-01  2013-06-30
10  2014-03-01  2014-06-30
11  2015-03-01  2015-06-30
12  2016-03-01  2016-06-30
13  2017-03-01  2017-06-30
14  2018-03-01  2018-06-30
15  2019-03-01  2019-06-30
16  2020-03-01  2020-06-30
17  2021-03-01  2021-06-30
18  2022-03-01  2022-06-30
19  2023-03-01  2023-06-30
20  2024-03-01  2024-06-30


In [28]:
# prompt: iterate df

for index, row in df.iterrows():

  # Access the start and end dates
  start_date = row['start_date']
  end_date = row['end_date']

  # Print the start and end dates
  # print(f"Start date: {start_date}")
  # print(f"End date: {end_date}")


Start date: 2004-03-01
End date: 2004-06-30
Start date: 2005-03-01
End date: 2005-06-30
Start date: 2006-03-01
End date: 2006-06-30
Start date: 2007-03-01
End date: 2007-06-30
Start date: 2008-03-01
End date: 2008-06-30
Start date: 2009-03-01
End date: 2009-06-30
Start date: 2010-03-01
End date: 2010-06-30
Start date: 2011-03-01
End date: 2011-06-30
Start date: 2012-03-01
End date: 2012-06-30
Start date: 2013-03-01
End date: 2013-06-30
Start date: 2014-03-01
End date: 2014-06-30
Start date: 2015-03-01
End date: 2015-06-30
Start date: 2016-03-01
End date: 2016-06-30
Start date: 2017-03-01
End date: 2017-06-30
Start date: 2018-03-01
End date: 2018-06-30
Start date: 2019-03-01
End date: 2019-06-30
Start date: 2020-03-01
End date: 2020-06-30
Start date: 2021-03-01
End date: 2021-06-30
Start date: 2022-03-01
End date: 2022-06-30
Start date: 2023-03-01
End date: 2023-06-30
Start date: 2024-03-01
End date: 2024-06-30


In [31]:
# Access the Landsat 8 imagery collection
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')


for index, row in df.iterrows():
  # Filter the collection by date
  image = image_collection.filterDate(row['start_date'], row['end_date']).filterBounds(roi).sort('CLOUD_COVER').first()
  image = image.clip(roi)

  landsat_rgb = image.select(['B4', 'B3', 'B2'])



In [32]:
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}


In [20]:
# prompt: write the code to display the "landsat_rgb" as the image

# Display the Landsat 8 image
Map = geemap.Map()
Map.addLayer(landsat_rgb, vis_params, 'Landsat 8 Image')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [17]:
# Create a Folium map object
map = folium.Map(location=[28.17, 82.37], zoom_start=10)

# Add the tile layer to the map
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    max_zoom=24
).add_to(map)

# Display the map
map
